In [40]:
# automatically reload the .py files being used for event selections and signal regions
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys # import
# # update the pip package installer
!{sys.executable} -m pip install --upgrade --user pip
# # install required packages
!{sys.executable} -m pip install --upgrade --user uproot awkward vector matplotlib

In [3]:
!{sys.executable} -m pip install --user numpy==2.0.2
!{sys.executable} -m pip install --user requests
!{sys.executable} -m pip install --user aiohttp

In [4]:
import sys #reimport sys so we have it when not running package installation/setup
import infofile # local file containing cross-sections, sums of weights, dataset IDs
import numpy as np # for numerical calculations such as histogramming
import matplotlib.pyplot as plt # for plotting
import matplotlib_inline # to edit the inline plot format

#suspicious this is necessary, never had to do it before!
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'svg') # to make plots in pdf (vector) format

from matplotlib.ticker import AutoMinorLocator # for minor ticks
import uproot # for reading .root files
import awkward as ak # to represent nested data in columnar format
import vector # for 4-momentum calculations
import time
MeV = 0.001
GeV = 1.0

In [41]:
# import modules
import sys
sys.path.append('.')  # Ensure current directory is in path

# import signal regions and preselection
# all of these take (data, channel)
from region_functions import (
    apply_preselection,
    signal_region,
    signal_region_VBF_new
)

# import specific functions
from selection_functions import (
    lep_type_selection,
    lep_charge_selection,
    lep_trigger_selection,
    lep_pt_selection,
    tight_selection,
    met_selection,
    calc_phi_dilepton,
    lepton_mll_selection,
    ptmiss_selection,
    calc_mass,
    calc_mT,
    calc_mt_ell,
    calc_mjj,
    calc_pT_ll,
    delta_phi,
    central_jet_veto,
    outside_lepton_veto,
    outside_lepton_veto,
    lepton_eta_selection,
    remove_jets_near_leptons,
    calc_rapidity,
    delta_y_jj,
    compute_pT_ll
)

In [6]:
# other imports
import os # for files
from uproot import open as uproot_open # for files
import json # for saving files
import gc # for memory issues

In [7]:
path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/" # file path
skims = ['4lep','3lep','2lep','1lep','1largeRjet1lep','1lep1tau','GamGam'] # will use 2lep for my analysis

# variables used 
variables = ['trigE', 'trigM', 'lep_pt', 'lep_phi', 'lep_type', 'lep_charge',
            'met_et', 'met_phi', 'jet_n', 'jet_MV2c10', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E',
             'lep_isTightID', 'lep_eta', 'lep_E', 'ditau_m',
            ]
# skim for 2 leptons
skim_2lep = skims[2] 

lumi = 10  # fb^-1 (total luminosity for ABCD combined)

# weight variables
weight_vars = ["mcWeight", "scaleFactor_PILEUP", "scaleFactor_ELE", "scaleFactor_MUON", "scaleFactor_LepTRIGGER"]

In [8]:
# MC samples used
samples = {
    'Zjets': {
        'list': ['Zee', 'Zmumu', 'Ztautau'],
        'color': "#8fd7d7",
        'legend': r'$Z+$jets'
    },
    'Wjets': {
        'list': ['Wplusenu', 'Wminusenu', 'Wplusmunu', 'Wminusmunu', 'Wplustaunu', 'Wminustaunu'],
        'color': "#00b0be",
        'legend': r'$W+$jets'
    },
    'diboson_leptonic': {
        'list': ['llll', 'lllv', 'llvv', 'lvvv'],
        'color': '#ff8c9a',
        'legend': 'Diboson (fully leptonic)'
    },
    'diboson_semileptonic': {
        'list': ['ZqqZll', 'WqqZll', 'WpqqWmlv', 'WplvWmqq', 'WlvZqq'],
        'color': "#f45f74",
        'legend': 'Diboson (semi-leptonic)'
    },
    'HWW': {
        'list': ['VBFH125_WW2lep', 'ggH125_WW2lep'],
        'color': "#ffcd8e",
        'legend': r'$H(125)\rightarrow WW \rightarrow lvlv$',
    },
    'top': {
        'list': ['ttbar_lep', 'single_top_tchan', 'single_antitop_tchan', 'single_top_wtchan', 'single_antitop_wtchan',
                 'single_top_schan', 'single_antitop_schan', 'ttW', 'ttee', 'ttmumu'],
        'color': "#ffb255",
        'legend': r'Top processes',
    }
}

# running on MC samples

In [42]:
# adjust these to go over different samples or different selections/no selections at all

#channels = ['0ggF', '1ggF', 'VBF', 'VBF_new'] # OR VBF_new 
channels = ['0ggF']
#channels = ['VBF_new']
#background_categories = ["Zjets", "Wjets", "diboson_leptonic", "top"] # MC samples
background_categories = ["Zjets", "Wjets", "diboson_leptonic", "top", "HWW", "diboson_semileptonic"] # MC samples
#background_categories = ['top']
#background_categories = ['HWW']
#background_categories = ["HWW", "diboson_semileptonic"]
selection_mode = "full"  # options: "nocuts", "preselection", "full"
region_type = "signal" 
output_dir = "background_json_by_selection_fixed_preselection_jets" # where files are saved


max_events = 50000 # max events
lumi = 10  # fb^-1
os.makedirs(output_dir, exist_ok=True)

# start the loop
for channel in channels:
    print(f"\n Channel: {channel}") # channels are the three different signal regions

    for bkg_group in background_categories: # loop through backgrounds
        print(f"\n Background group: {bkg_group}")
        grouped_data = {} # initialize

        # my kernel keeps crashing for these specific backgrounds... decrease the step size
        step_size = 250 if bkg_group in ["Wjets", "diboson_leptonic", "diboson_semileptonic", "HWW"] else 1000

        # proper weight procedure (following example notebook from class)
        for sample in samples[bkg_group]["list"]:
            print(f" Sample: {sample}")
            dsid = infofile.infos[sample]["DSID"]
            xsec = infofile.infos[sample]["xsec"]
            sumw = infofile.infos[sample]["sumw"]
            red_eff = infofile.infos[sample]["red_eff"]

            # now accessing the file
            file_path = f"{path}{skim_2lep}/MC/mc_{dsid}.{sample}.{skim_2lep}.root"
            tree = uproot_open(file_path + ":mini")
            total_events = tree.num_entries
            # scale depending on how many events are being used
            sample_scale = total_events / min(total_events, max_events) # however many events are used
            xsec_weight = sample_scale * (lumi * 1000 * xsec) / (sumw * red_eff) # get the weight

            sample_events = [] # initialize for loop

            # weight and data
            for data_w, data in zip(
                tree.iterate(weight_vars, library="pd", step_size=step_size, entry_stop=max_events),
                tree.iterate(variables, library="ak", step_size=step_size, entry_stop=max_events)
            ):

                # derived quantities using functions
                data["dilepton_phi"] = calc_phi_dilepton(data["lep_pt"], data["lep_eta"], data["lep_phi"], data["lep_E"]) # phi for dilepton system
                data["delta_phi_ll_met"] = delta_phi(data["dilepton_phi"], data["met_phi"]) # angle between dilepton system and MET
                data["osof_mass"] = calc_mass(data["lep_pt"], data["lep_eta"], data["lep_phi"], data["lep_E"]) # dilepton invariant mass
                data["mT"] = calc_mT(data["met_et"], data["met_phi"], data["lep_pt"], data["lep_phi"], data["osof_mass"]) # dilepton transverse mass
                data["mT_ell"] = calc_mt_ell(data["lep_pt"], data["lep_phi"], data["met_et"], data["met_phi"]) # MAX transverse mass for lepton
                data["pT_ll"] = compute_pT_ll(data["lep_pt"], data["lep_phi"]) # dilepton transverse momentum
                data["delta_phi_ll"] = delta_phi(data["lep_phi"][:, 0], data["lep_phi"][:, 1]) # angle between the two leptons

                #if only making preselections
                # if selection_mode == "preselection": # if only choosing preselection cuts
                #     data = apply_preselection(data, channel)
                #     # move on if empty
                #     if len(data) == 0:
                #         continue
                #     # weights
                #     data_w = data_w.iloc[:len(data)] # not correct here

                # if applying full SR selections
                if selection_mode == "full": # if doing full SR cuts
                    data, preselection_mask = apply_preselection(data, channel)
                    #print(f"After preselection: {len(data)} events")
                    # move on if empty
                    if len(data) == 0:
                        continue
                    # weights
                    # incorrect:
                    #data_w = data_w.iloc[:len(data)] # NOT INDEXING PROPERLY
                    # issue with mc weights here, I was just matching the lengths of these, not the proper indices
                    data_w = data_w.iloc[ak.to_numpy(preselection_mask)] # properly apply mask to weights
            

                    # if using the new VBF SR
                    if channel == 'VBF_new':
                        region_mask = signal_region_VBF_new(data, channel) if region_type == "signal" else ak.Array([False] * len(data))
                        data = data[region_mask] # apply mask
                        data_w = data_w.iloc[ak.to_numpy(region_mask)] # weights
                        if len(data) == 0:
                            continue
                    else:
                        # otherwise use other specified signal region
                        region_mask = signal_region(data, channel) if region_type == "signal" else ak.Array([False] * len(data))
                        data = data[region_mask]
                        data_w = data_w.iloc[ak.to_numpy(region_mask)]
                        if len(data) == 0:
                            continue

                # compute mjj if looking at VBF
                if channel == "VBF":
                    data["mjj"] = calc_mjj(data["jet_pt"], data["jet_eta"], data["jet_phi"], data["jet_E"]) # dijet invariant mass

                # compute mjj and delta_y_jj for new VBF SR and apply cuts
                if channel == "VBF_new":
                    data["mjj"] = calc_mjj(data["jet_pt"], data["jet_eta"], data["jet_phi"], data["jet_E"]) # dijet invariant mass
                    #print(data['mjj'])
                    #data["delta_y_jj"] = delta_y_jj(data["jet_pt"], data["jet_eta"], data["jet_phi"], data["jet_E"])
                    data["delta_y_jj"] = ak.Array([
                                                    delta_y_jj({
                                                        "jet_pt": data["jet_pt"][i], # jet pT
                                                        "jet_eta": data["jet_eta"][i], # jet eta
                                                        "jet_E": data["jet_E"][i] # jet energy
                                                    }) if len(data["jet_pt"][i]) >= 2 else np.nan
                                                    for i in range(len(data))
                                                ]) # difference in rapidity between the two jets
                    #print(data['delta_y_jj'])
                    #mjj_cut = (data['mjj'] > 400000) # apply mjj > 400 GeV
                    mjj_cut = (data['mjj'] > 350000) # apply mjj > 350 GeV
                    #print('mjj cut', mjj_cut)
                    delta_y_jj_cut = np.abs(data['delta_y_jj'] > 2) # abs delta_y_jj > 1.5
                    #print('dYjj cut', delta_y_jj_cut)
                    combined_cut = mjj_cut & delta_y_jj_cut # mask combining these selections
                    data = data[combined_cut] # apply the cut
                    data_w = data_w.iloc[ak.to_numpy(combined_cut)] # weights
                    if len(data) == 0: # if empty move on
                        continue
                    

                # for events with jets
                if "jet_pt" in data.fields:
                    jets_over_30 = data["jet_pt"] > 30000 # count number of jets with pT > 30 GeV
                    data["Njets"] = ak.sum(jets_over_30, axis=1)
                else: # if jets not in the file
                    data["Njets"] = ak.zeros_like(data["lep_pt"][:, 0], dtype=int)

                # total weight
                total_weight = data_w[weight_vars].prod(axis=1) * xsec_weight

                # now store variables that we computed or were in the file to begin with
                for i in range(len(data)):
                    event = {
                        "delta_phi_ll_met": float(data["delta_phi_ll_met"][i]), # difference in angle between dilepton system and MET
                        "delta_phi_ll": float(data["delta_phi_ll"][i]), # angle between leptons
                        "mT": float(data["mT"][i]), # dilepton transverse mass
                        "mT_ell": float(data["mT_ell"][i]), # max of lepton transverse mass
                        "met_et": float(data["met_et"][i]), # MET
                        "met_phi": float(data["met_phi"][i]), # MET phi
                        "osof_mass": float(data["osof_mass"][i]), # dilepton invariant mass
                        "lep_pt": [float(x) for x in data["lep_pt"][i]], # lepton pT
                        "lep_eta": [float(x) for x in data["lep_eta"][i]], # lepton eta
                        "lep_phi": [float(x) for x in data["lep_phi"][i]], # lepton phi
                        "lep_E": [float(x) for x in data["lep_E"][i]], # lepton energy
                        "weight": float(total_weight.iloc[i]), # weights
                        "Njets": int(data["Njets"][i]), # number of jets
                        "pT_ll": float(data["pT_ll"][i]), # dilepton transverse momentum
                        "dilepton_phi": float(data["dilepton_phi"][i]) # phi of dilepton system
                    }
                    # for VBF signal regions (involving jet variables)
                    if channel in ("VBF", "VBF_new") and "jet_pt" in data.fields:
                        jets = {
                            "jet_pt": [float(x) for x in data["jet_pt"][i]], # transverse momentum
                            "jet_eta": [float(x) for x in data["jet_eta"][i]], # jet eta
                            "jet_phi": [float(x) for x in data["jet_phi"][i]], # azimuthal angle
                            "jet_E": [float(x) for x in data["jet_E"][i]], # jet energy
                            "mjj": float(data["mjj"][i]) # dijet invariant mass
                        }
                        event.update(jets)

                    # append that event with the variables saved
                    sample_events.append(event)

            grouped_data[sample] = sample_events
            del tree, data, data_w, sample_events # delete for memory help
            gc.collect()

        # save to json file for the group
        save_name = f"{bkg_group}_{channel}_{selection_mode}.json"
        with open(os.path.join(output_dir, save_name), "w") as f:
            json.dump({
                "samples": grouped_data, # save the samples
                "color": samples[bkg_group]["color"], # colour it has
                "legend": samples[bkg_group]["legend"] # title
            }, f, indent=2)
        print(f"Saved: {save_name}")
        del grouped_data # for memory issues
        gc.collect()


 Channel: 0ggF

 Background group: Zjets
 Sample: Zee
 Sample: Zmumu
 Sample: Ztautau
Saved: Zjets_0ggF_full.json

 Background group: Wjets
 Sample: Wplusenu
 Sample: Wminusenu
 Sample: Wplusmunu
 Sample: Wminusmunu
 Sample: Wplustaunu
 Sample: Wminustaunu
Saved: Wjets_0ggF_full.json

 Background group: diboson_leptonic
 Sample: llll
 Sample: lllv
 Sample: llvv
 Sample: lvvv
Saved: diboson_leptonic_0ggF_full.json

 Background group: top
 Sample: ttbar_lep
 Sample: single_top_tchan
 Sample: single_antitop_tchan
 Sample: single_top_wtchan
 Sample: single_antitop_wtchan
 Sample: single_top_schan
 Sample: single_antitop_schan
 Sample: ttW
 Sample: ttee
 Sample: ttmumu
Saved: top_0ggF_full.json

 Background group: HWW
 Sample: VBFH125_WW2lep
 Sample: ggH125_WW2lep
Saved: HWW_0ggF_full.json

 Background group: diboson_semileptonic
 Sample: ZqqZll
 Sample: WqqZll
 Sample: WpqqWmlv
 Sample: WplvWmqq
 Sample: WlvZqq
Saved: diboson_semileptonic_0ggF_full.json


# running on data samples

In [24]:
#channels = ["0ggF", "1ggF"]#, "VBF", "VBF_new"]
#channels = ['1ggF']
channels = ['0ggF']
samples_data = {
    'data': {
        'list': ['data_A', 'data_B', 'data_C', 'data_D']
    }
}
selection_mode = "full"  #  "preselection", "full", "nocuts"
region_type = "signal"
output_dir = "data_json_by_selection_test_FULL_0ggF_new" # where the json files are saved
max_events = 6000000
step_size = 600000
os.makedirs(output_dir, exist_ok=True) # will make the directory if it doesn't exist

for channel in channels: # loop through channels
    print(f"\n channel: {channel}")
    period_data = {"A": [], "B": [], "C": [], "D": []} # all data files

    for sample in samples_data["data"]["list"]: # go through the files
        print(f"Data Sample: {sample}")
        period = sample.split("_")[-1]
        assert period in period_data
        # get file path
        file_path = path + skim_2lep + "/Data/" + sample + "." + skim_2lep + ".root"
        tree = uproot_open(file_path + ":mini") # open the file using uproot

        # go through the data
        for data in tree.iterate(variables, library="ak", step_size=step_size, entry_stop=max_events):
            data["dilepton_phi"] = calc_phi_dilepton(data["lep_pt"], data["lep_eta"], data["lep_phi"], data["lep_E"]) # phi of the two leptons
            data["delta_phi_ll_met"] = delta_phi(data["dilepton_phi"], data["met_phi"]) # difference in angle between the dilepton system and MET
            data["osof_mass"] = calc_mass(data["lep_pt"], data["lep_eta"], data["lep_phi"], data["lep_E"]) # dilepton invariant mass
            data["mT"] = calc_mT(data["met_et"], data["met_phi"], data["lep_pt"], data["lep_phi"], data["osof_mass"]) # dilepton transverse mass
            data["mT_ell"] = calc_mt_ell(data["lep_pt"], data["lep_phi"], data["met_et"], data["met_phi"]) # max of lepton transverse mass
            data["pT_ll"] = compute_pT_ll(data["lep_pt"], data["lep_phi"]) # dilepton transverse momentum
            data["delta_phi_ll"] = delta_phi(data["lep_phi"][:, 0], data["lep_phi"][:, 1]) # difference in phi between the two leptons

        # apply preselection if using preselection or full
            if selection_mode in ["preselection", "full"]:
                data = apply_preselection(data, channel)
                if len(data) == 0: # skip if no events remain
                    continue

        # if applying full SR
            if selection_mode == "full":
                if channel == "VBF_new": # check case for VBF new (added)
                    region_mask = signal_region_VBF_new(data, channel) if region_type == "signal" else ak.Array([False] * len(data))
                    data = data[region_mask] # apply mask
                    if len(data) == 0: # skip if empty
                        continue
                    # compute mjj and delta_y_jj
                    data["mjj"] = calc_mjj(data["jet_pt"], data["jet_eta"], data["jet_phi"], data["jet_E"])
                    data["delta_y_jj"] = ak.Array([
                        delta_y_jj({
                            "jet_pt": data["jet_pt"][i], # jet pT
                            "jet_eta": data["jet_eta"][i], # jet eta
                            "jet_E": data["jet_E"][i] # jet energy
                        }) if len(data["jet_pt"][i]) >= 2 else np.nan
                        for i in range(len(data))
                    ])
                    # apply cut for mjj > 400 GeV and abs(delta_y)jj) > 2
                    mjj_cut = (data['mjj'] > 350000) # lower to 350 GeV
                    delta_y_jj_cut = np.abs(data['delta_y_jj'] > 2)
                    combined_cut = mjj_cut & delta_y_jj_cut
                    data = data[combined_cut] # mask for events passing
                    if len(data) == 0:
                        continue
                else: # if not new VBF SR, use the selected SR
                    region_mask = signal_region(data, channel) if region_type == "signal" else ak.Array([False] * len(data))
                    data = data[region_mask] # apply SR mask
                    if len(data) == 0:
                        continue

            if channel == "VBF":  # look at mjj if using VBF
                data["mjj"] = calc_mjj(data["jet_pt"], data["jet_eta"], data["jet_phi"], data["jet_E"])
           
            # check number of jets with pT > 30 GeV for events with jets
            if "jet_pt" in data.fields:
                jets_over_30 = data["jet_pt"] > 30000
                data["Njets"] = ak.sum(jets_over_30, axis=1)
            else: # return 0 if not
                data["Njets"] = ak.zeros_like(data["lep_pt"][:, 0], dtype=int)
            # now store the calculated variables and the variables from the file
            for i in range(len(data)):
                event = {
                    "delta_phi_ll_met": float(data["delta_phi_ll_met"][i]), # angle between dilepton system and MET
                    "delta_phi_ll": float(data["delta_phi_ll"][i]), # angle between the two leptons
                    "mT": float(data["mT"][i]), # dilepton transverse mass
                    "mT_ell": float(data["mT_ell"][i]), # max of lepton transverse mass
                    "met_et": float(data["met_et"][i]), # MET
                    "met_phi": float(data["met_phi"][i]), # met phi
                    "osof_mass": float(data["osof_mass"][i]), # dilepton invariant mass
                    "lep_pt": [float(x) for x in data["lep_pt"][i]], # lepton pT
                    "lep_eta": [float(x) for x in data["lep_eta"][i]], # lepton eta
                    "lep_phi": [float(x) for x in data["lep_phi"][i]], # lepton phi
                    "lep_E": [float(x) for x in data["lep_E"][i]], # lepton energy
                    "Njets": int(data["Njets"][i]), # number of jets
                    "pT_ll": float(data["pT_ll"][i]), # dilepton transverse momentum
                    "dilepton_phi": float(data["dilepton_phi"][i]) # phi of dilepton system
                }
                # if looking at VBF SRs, store the jet variables
                if channel in ("VBF", "VBF_new") and "jet_pt" in data.fields:
                    jets = {
                        "jet_pt": [float(x) for x in data["jet_pt"][i]], # jet pT
                        "jet_eta": [float(x) for x in data["jet_eta"][i]], # jet energy
                        "jet_phi": [float(x) for x in data["jet_phi"][i]], # jet phi
                        "jet_E": [float(x) for x in data["jet_E"][i]], # jet energy
                        "mjj": float(data["mjj"][i]) # dijet invariant mass
                    }
                    event.update(jets)

                # store for the period
                period_data[period].append(event)

        del tree, data # delete for memory issues
        gc.collect()

    # save output to a json file
    save_path = os.path.join(output_dir, f"data_{channel}_{selection_mode}.json")
    with open(save_path, "w") as f: # open a new file and save to it
        json.dump({"samples": period_data}, f, indent=2)
    print(f"Saved: {save_path}")


 channel: 0ggF
Data Sample: data_A
Data Sample: data_B
Data Sample: data_C
Data Sample: data_D
Saved: data_json_by_selection_test_FULL_0ggF_new\data_0ggF_full.json


In [14]:
# to check how many events are in the data files
channels = ["0ggF"]
# data A to D
samples_data = {
    'data': {
        'list': ['data_A', 'data_B', 'data_C', 'data_D']
    }
}
skim_2lep = "2lep" # using 2 lepton skim
path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/" # path for the files
for channel in channels: # loop through
    print(f"channel {channel}")
    for sample in samples_data["data"]["list"]:
        period = sample.split("_")[-1]
        file_path = path + skim_2lep + "/Data/" + sample + "." + skim_2lep + ".root" # get the file path
        try: # try to open if proper file path
            tree = uproot_open(file_path + ":mini")
            print(f" - {sample} ({period}): {tree.num_entries} events")
        except Exception as e: # if path is messed up
            print(f"file issue/wrong file")

channel 0ggF
 - data_A (A): 668152 events
 - data_B (B): 2459370 events
 - data_C (C): 3587872 events
 - data_D (D): 5490396 events


----------

weight info from sample notebook:


Additionally,
    there is a cross-section weight $w_\sigma$ associated with each MC file.
We define this variable `xsec_weight` below. 
This weight is meant to normalise the entire Monte-Carlo distribution based on the number of events in the data.
This is its definition:
$$ w_\sigma = \frac{\int L \text{d}t ~ \sigma }{\eta \sum_i w_i } $$
where $\int L \text{d}t$ is the integrated luminosity (`lumi`),
    $\sigma$ is the cross section (`info["xsec"]`),
    $\eta$ is the filter efficiency of the MC generator,
    and $\sum_i w_i$ gives the sum of all weights (`info["sumw"]`).
When the integrated luminosity is multiplied by the cross section,
    it gives a measure of the total number of events during a period of data taking.

In [27]:
# testing the overlap removal
# testing the overlap function first:
# sample arrays
jet_eta = ak.Array([[0.5, -1.1], [1.2, -2.0, 0.3]])
jet_phi = ak.Array([[0.1, -0.1], [1.0, -1.5, 0.2]])
lep_eta = ak.Array([[1.1, -0.3], [0.4, -1.7]])
lep_phi = ak.Array([[0.0, 0.3], [-1.2, 0.8]])

# check overlap removal
keep = remove_jets_near_leptons(jet_eta, jet_phi, lep_eta, lep_phi, deltaR=0.2)
print(keep)

[[True, True], [True, True, True]]


In [29]:
# check mask
filtered_pt = jet_eta[keep]
print(filtered_pt)
# keeps all of them

[[0.5, -1.1], [1.2, -2, 0.3]]


In [35]:
# Sample data: two events
data = {
    "jet_pt": ak.Array([[40_000, 20_000], [60_000, 50_000, 30_000]]),  # 2 jets in event 0, 3 jets in event 1
    "jet_eta": ak.Array([[0.5, -0.8], [1.2, -2.0, 0.3]]),
    "jet_phi": ak.Array([[0.1, -0.1], [1.0, -1.5, 0.2]]),
    "jet_E": ak.Array([[50_000, 30_000], [70_000, 60_000, 40_000]]),
    "jet_MV2c10": ak.Array([[0.1, 0.2], [0.3, 0.85, 0.4]])
}

# sample mask - want to remove 2nd jet in the first event, keep all jets in the second event
keep_mask = ak.Array([[True, False], [True, True, True]])

# check condition
if not ak.all(keep_mask):
    print("apply filter")
    jet_keys = ["jet_pt", "jet_eta", "jet_phi", "jet_E", "jet_MV2c10"]
    data.update({key: data[key][keep_mask] for key in jet_keys})

# check output
for i in range(len(data["jet_pt"])):
    print(f"\nEvent {i}")
    for key in data:
        print(f"{key}: {data[key][i]}")

apply filter

Event 0
jet_pt: [40000]
jet_eta: [0.5]
jet_phi: [0.1]
jet_E: [50000]
jet_MV2c10: [0.1]

Event 1
jet_pt: [60000, 50000, 30000]
jet_eta: [1.2, -2, 0.3]
jet_phi: [1, -1.5, 0.2]
jet_E: [70000, 60000, 40000]
jet_MV2c10: [0.3, 0.85, 0.4]


In [ ]:
# removed the second jet in the first event and kept all jets in the second event